In [1]:
import openpyxl
import pandas as pd
from datetime import datetime, timedelta
import shutil
import os
import sys 
import time

In [31]:
def get_workfile(path):
    workbook_path = path
    workbook = openpyxl.load_workbook(workbook_path)
    sheet = workbook['银行卡明细查询模板']
    return workbook, sheet

In [32]:
def date_split(start_time, end_time):
    time_periods = []
    current_time = start_time
    while current_time < end_time:
        next_time = current_time + timedelta(days=85)
        if next_time > end_time:
            next_time = end_time
        time_periods.append((current_time, next_time))
        current_time = next_time
    return time_periods

In [33]:
class Constants:
    MainCategory_0 = '个人'
    MainCategory_1 = '对公'
    SplitMethod = '按月拆分'

In [34]:
class Card:  
    def __init__(self, name, card_number, bank, start_time, end_time):  
        self.main_category = Constants.MainCategory_0 
        self.name = name  
        self.card_number = card_number  
        self.bank = bank  
        self.start_time = datetime.strptime(start_time.split(' ')[0], '%Y-%m-%d')  
        self.end_time = datetime.strptime(end_time.split(' ')[0], '%Y-%m-%d') 
        self.split_method = Constants.SplitMethod
  
    def get_main_category(self):  
        return self.main_category  
  
    def set_main_category(self, main_category):  
        self.main_category = main_category  
  
    def get_name(self):  
        return self.name  
  
    def set_name(self, name):  
        self.name = name  
  
    def get_card_number(self):  
        return self.card_number  
  
    def set_card_number(self, card_number):  
        self.card_number = card_number  
  
    def get_bank(self):  
        return self.bank  
  
    def set_bank(self, bank):  
        self.bank = bank  
  
    def get_start_time(self):  
        return self.start_time  
  
    def set_start_time(self, start_time):  
        self.start_time = start_time  
  
    def get_end_time(self):  
        return self.end_time  
  
    def set_end_time(self, end_time):  
        self.end_time = end_time  
  
    def get_split_method(self):  
        return self.split_method  
  
    def set_split_method(self, split_method):  
        self.split_method = split_method

In [35]:
def wirite_paperwork(card_list, template_path, output_dir):
    file_index = 1
    row_index = 3

    workbook, sheet = get_workfile(template_path)

    for card in card_list:
        for date in date_split(card.get_start_time(), card.get_end_time()):        
            sheet.cell(row=row_index, column=1, value=card.get_main_category())
            sheet.cell(row=row_index, column=2, value=card.get_name())
            sheet.cell(row=row_index, column=3, value=card.get_card_number())
            sheet.cell(row=row_index, column=4, value=card.get_bank())
            sheet.cell(row=row_index, column=5, value=date[0].strftime("%Y-%m-%d"))
            sheet.cell(row=row_index, column=6, value=date[1].strftime("%Y-%m-%d"))
            sheet.cell(row=row_index, column=7, value=card.get_split_method())
            row_index += 1

            if row_index > 195:
                workbook.save(f'{output_dir}/zn_{file_index}.xlsx')
                file_index += 1
                workbook.close()
                workbook, sheet = get_workfile(template_path)
                row_index = 3

    workbook.save(f'{output_dir}/zn_{file_index}.xlsx')
    workbook.close()

In [36]:
# bank_path = input('请输入银行卡调单文件（姓名,对手账户,发卡机构,开始时间,结束时间）路径：')
bank_path = '../填写模板.xlsx'

# BASE_DIR = os.path.dirname(os.path.realpath(sys.argv[0]))
BASE_DIR = r'C:\Users\18428\Documents\代码\RPA程序\银行丢单表填写v2'
SOURCES_DIR = os.path.join(BASE_DIR, 'sources')

bank_dict_path = os.path.join(SOURCES_DIR, '发卡机构字典.xlsx')
template_path = os.path.join(SOURCES_DIR, 'zn_template.xlsx')   # 治安调单表模板
output_dir = os.path.join(BASE_DIR, 'output')
tmp_dir = os.path.join(BASE_DIR, 'tmp')

In [37]:
bank_df = pd.read_excel(bank_path, dtype=str)
bank_dict = pd.read_excel(bank_dict_path, dtype=str)
merged_df = pd.merge(bank_df, bank_dict, how='left', on='发卡机构')
merged_df['治安发卡机构'].fillna('-', inplace=True)

t_df = merged_df[['姓名', '卡号', '发卡机构', '治安发卡机构']]
merged_df = merged_df[merged_df['治安发卡机构'] != '-']

t_df = t_df[t_df['治安发卡机构'] == '-']
t_df.drop_duplicates(inplace=True, subset=['发卡机构'])
t_df.to_excel(os.path.join(BASE_DIR, '调单卡_未匹配到.xlsx'), index=False)
if t_df.empty:  
    print("开户行已全部匹配")  
else:  
    print('未匹配数据：')
    print(t_df.head())

未匹配数据：
     姓名                   卡号         发卡机构 治安发卡机构
20  杨鑫鑫  6231090401001532056   天津滨海农村商业银行      -
25  陈顺峰     6223190218900063         青岛农信      -
31  徐鑫淼  6235310001004692421   郑州银行股份有限公司      -
42  张俊羊  6230270600010163878  陕西省农村信用社联合社      -
44  吴甜甜   622960868160889803      邯郸市商业银行      -


In [38]:
merged_df = merged_df[['姓名', '卡号', '治安发卡机构', '开始时间', '结束时间']].values
card_list = [Card(bl[0], bl[1], bl[2], bl[3], bl[4]) for bl in merged_df]

In [39]:
if not os.path.exists(tmp_dir):  
        os.mkdir(tmp_dir)  
else:
    shutil.rmtree(tmp_dir)
    os.mkdir(tmp_dir)
    
if not os.path.exists(output_dir):  
        os.mkdir(output_dir)  
else:
    shutil.rmtree(output_dir)
    os.mkdir(output_dir)

In [40]:
wirite_paperwork(card_list, template_path, tmp_dir)

In [41]:
import win32com.client # 调用wps接口

xlsx_path = os.listdir(tmp_dir)

for p in xlsx_path:
    path = os.path.join(tmp_dir, p)
    print(path)
    application = win32com.client.Dispatch('Excel.Application')
    workbook = application.Workbooks.Open(path)
    worksheet = workbook.Worksheets('银行卡明细查询模板')
    dirname, filename = os.path.split(path)
    worksheet.SaveAs(os.path.join(output_dir, filename))
    workbook.Close()
    application.Quit()
shutil.rmtree(tmp_dir)
print('运行结束3秒后关闭')
# time.sleep(3)

C:\Users\18428\Documents\代码\RPA程序\银行丢单表填写v2\tmp\zn_1.xlsx
C:\Users\18428\Documents\代码\RPA程序\银行丢单表填写v2\tmp\zn_2.xlsx
C:\Users\18428\Documents\代码\RPA程序\银行丢单表填写v2\tmp\zn_3.xlsx
C:\Users\18428\Documents\代码\RPA程序\银行丢单表填写v2\tmp\zn_4.xlsx
C:\Users\18428\Documents\代码\RPA程序\银行丢单表填写v2\tmp\zn_5.xlsx
C:\Users\18428\Documents\代码\RPA程序\银行丢单表填写v2\tmp\zn_6.xlsx
C:\Users\18428\Documents\代码\RPA程序\银行丢单表填写v2\tmp\zn_7.xlsx
运行结束3秒后关闭
